# HW 2 - Carlos Alvarado

In [3]:
import sys
sys.path.append('../source')

from helper_functions import read_data, tabular

data = read_data('data/credit-data.csv')

data.columns


Index(['PersonID', 'SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines',
       'age', 'zipcode', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio',
       'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans',
       'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines',
       'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents'],
      dtype='object')

In [4]:
tabular(data, 'zipcode')

<bound method DataFrame.sort of        zipcode   Percent
60625    33514  0.223427
60629    16840  0.112267
60637    16625  0.110833
60657    16624  0.110827
60618    16612  0.110747
60804    16605  0.110700
60601    16599  0.110660
60644    16581  0.110540>
